# Deep Learning

In [1]:
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras.layers.core import Dropout
from keras import regularizers
import keras
import pandas as pd
import numpy as np
from keras import backend as K
from keras import metrics
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

Using TensorFlow backend.


In [2]:
kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")

#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
#y_test_labels = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")

output_columns_2labels = ['is_Attack','is_Normal']

from sklearn import model_selection as ms
from sklearn import preprocessing as pp

x_input = kdd_train_2labels.drop(output_columns_2labels, axis = 1)
y_output = kdd_train_2labels.loc[:,output_columns_2labels]

ss = pp.StandardScaler()
x_input = ss.fit_transform(x_input)

#le = pp.LabelEncoder()
#y_train = le.fit_transform(y_train_labels).reshape(-1, 1)
#y_test = le.transform(y_test_labels).reshape(-1, 1)

y_train = kdd_train_2labels.loc[:,output_columns_2labels].values

x_train, x_valid, y_train, y_valid = ms.train_test_split(x_input, 
                              y_train, 
                              test_size=0.1)
#x_valid, x_test, y_valid, y_test = ms.train_test_split(x_valid, y_valid, test_size = 0.4)

x_test = kdd_test_2labels.drop(output_columns_2labels, axis = 1)
y_test = kdd_test_2labels.loc[:,output_columns_2labels].values

x_test = ss.transform(x_test)

#x_train = np.hstack((x_train, y_train))
#x_valid = np.hstack((x_valid, y_valid))

#x_test = np.hstack((x_test, np.random.normal(loc = 0, scale = 0.01, size = y_test.shape)))

In [3]:
input_dim = 122
intermediate_dim = 10
latent_dim = 32
batch_size = 1409
hidden_layers = 8
classes = 2
drop_prob = 0.2

class Train:
    def build_vae_model():
        Train.x = Input(shape=(input_dim,))
        
        hidden_encoder = Train.x
        for i in range(hidden_layers):
            hidden_encoder = Dense(intermediate_dim, activation='relu', 
                      kernel_regularizer=keras.regularizers.l2(0.0001),
                      activity_regularizer=keras.regularizers.l1(0.0001))(hidden_encoder)
            
            hidden_encoder = Dropout(rate=drop_prob)(hidden_encoder)

        Train.mean_encoder = Dense(latent_dim, activation=None)(hidden_encoder)
        Train.logvar_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        def get_distrib(args):

            m_e, l_e = args

            # Sample epsilon
            epsilon = np.random.normal(loc=0.0, scale=0.05, size = (batch_size, latent_dim))

            # Sample latent variable
            z = m_e + K.exp(l_e / 2) * epsilon
            return z

        z = Lambda(get_distrib,name='z_dist')([Train.mean_encoder, Train.logvar_encoder])

        hidden_decoder = z
        for i in range(hidden_layers):
            hidden_decoder = Dense(intermediate_dim, activation="relu", 
                      kernel_regularizer=keras.regularizers.l2(0.0001),
                      activity_regularizer=keras.regularizers.l1(0.0001))(hidden_decoder)
            hidden_decoder = Dropout(rate=drop_prob)(hidden_decoder)

        Train.x_ = Dense(input_dim, activation=None, name='vae_output')(hidden_decoder)
        
    def build_softmax_model():
        Train.z_ = Input(shape=(latent_dim,))
        hidden_y = Dense(latent_dim, activation='softmax', name='softmax_hidden')(Train.z_)
        Train.y = Dense(classes, activation='softmax', name='softmax_output')(hidden_y)
        
def vae_loss(x, x_decoded_mean):
    xent_loss = input_dim * keras.losses.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + Train.logvar_encoder - K.square(Train.mean_encoder) - K.exp(Train.logvar_encoder), axis=-1)
    return K.mean(xent_loss + kl_loss)


Train.build_vae_model()
Train.build_softmax_model()


In [4]:
import itertools
#features_arr = [4, 16, 32, 256, 1024]
#hidden_layers_arr = [2, 6, 10, 100]

#features_arr = [4, 16, 32]
#hidden_layers_arr = [2, 6, 10]

features_arr = [4, 16, 32]
hidden_layers_arr = [2, 4, 6]

epoch_arr = [5]

score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
scores = []
predictions = {}

for e, h, f in itertools.product(epoch_arr, hidden_layers_arr, features_arr):
    
    print(" \n Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
    latent_dim = f
    epochs = e
    hidden_layers = h
    
    train_size = x_train.shape[0] - x_train.shape[0]%batch_size
    valid_size = x_valid.shape[0] - x_valid.shape[0]%batch_size

    
    optimizer = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-04, decay=0.1)
    
    vae_model = Model(inputs = Train.x, outputs = Train.x_)
    vae_model.compile(optimizer = optimizer, 
                      loss = vae_loss)    
    vae_model.fit(x = x_train[:train_size,:], y = x_train[:train_size,:], 
                  shuffle=True, epochs=epochs, 
                  batch_size = batch_size, 
                  validation_data = (x_test, x_test),
                  verbose = 1)
    
    z_model = Model(inputs = Train.x, outputs = vae_model.get_layer("z_dist").output)
    z_train = z_model.predict(x_train[:train_size,:], batch_size=batch_size)
    z_valid = z_model.predict(x_valid[:valid_size,:], batch_size=batch_size)
    z_test = z_model.predict(x_test, batch_size=batch_size)
    
    sm_model = Model(inputs = Train.z_, outputs = Train.y)
    sm_model.compile(optimizer = optimizer, 
                      loss = keras.losses.categorical_crossentropy, 
                      metrics = ['accuracy'])
    
    sm_model.fit(x = z_train, y = y_train[:train_size,:],
                 shuffle=True, epochs=5, 
                  batch_size = batch_size, 
                  validation_data = (z_test, y_test),
                  verbose = 1)

    
    score_train = sm_model.evaluate(z_valid, y = y_valid[:valid_size,:],
                               batch_size = batch_size,
                               verbose = 1)
    
    score_test = sm_model.evaluate(z_test, y = y_test,
                           batch_size = batch_size,
                           verbose = 1)
    
    y_test_pred = sm_model.predict(z_test, batch_size=batch_size)
    

    y_pred = y_test_pred #np.argmax(y_test_pred[:,-2:], axis = 1)
    
    curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,0], "Normal_prob":y_pred[:,1]})
    predictions.update({"{}_{}_{}".format(e,f,h):curr_pred})
    
    scores.append(score(e,f,h,score_train[-1], score_test[-1])) #score_test[-1]))
    
    print("\n Train Acc: {}, Test Acc: {}".format(score_train[-1], 
                                                  score_test[-1])  )
    
scores = pd.DataFrame(scores)

 
 Current Layer Attributes - epochs:5 hidden layers:2 features count:4
Train on 112720 samples, validate on 22544 samples
Epoch 1/5
112720/112720 [==============================] - 4s - loss: -2.0037 - val_loss: 16.1710
Epoch 2/5
112720/112720 [==============================] - 1s - loss: 1079.3023 - val_loss: 17.4877
Epoch 3/5
112720/112720 [==============================] - 1s - loss: -3.6773 - val_loss: 18.3376
Epoch 4/5
112720/112720 [==============================] - 1s - loss: 92552.9985 - val_loss: 18.9935
Epoch 5/5
112720/112720 [==============================] - 1s - loss: -4.0252 - val_loss: 19.1719
Train on 112720 samples, validate on 22544 samples
Epoch 1/5
112720/112720 [==============================] - 0s - loss: 0.6948 - acc: 0.4659 - val_loss: 0.6910 - val_acc: 0.5693
Epoch 2/5
112720/112720 [==============================] - 0s - loss: 0.6948 - acc: 0.4659 - val_loss: 0.6910 - val_acc: 0.5692
Epoch 3/5
112720/112720 [==============================] - 0s - loss: 0.694

Train on 112720 samples, validate on 22544 samples
Epoch 1/5
112720/112720 [==============================] - 3s - loss: nan - val_loss: nan
Epoch 2/5
112720/112720 [==============================] - 1s - loss: nan - val_loss: nan
Epoch 3/5
112720/112720 [==============================] - 1s - loss: nan - val_loss: nan
Epoch 4/5
112720/112720 [==============================] - 1s - loss: nan - val_loss: nan
Epoch 5/5
112720/112720 [==============================] - 1s - loss: nan - val_loss: nan
Train on 112720 samples, validate on 22544 samples
Epoch 1/5
112720/112720 [==============================] - 1s - loss: nan - acc: 0.4659 - val_loss: nan - val_acc: 0.5692
Epoch 2/5
112720/112720 [==============================] - 0s - loss: nan - acc: 0.4659 - val_loss: nan - val_acc: 0.5692
Epoch 3/5
112720/112720 [==============================] - 0s - loss: nan - acc: 0.4659 - val_loss: nan - val_acc: 0.5692
Epoch 4/5
112720/112720 [==============================] - 0s - loss: nan - acc: 0

In [5]:
scores.sort_values("test_score", ascending=False)

,epoch,no_of_features,hidden_layers,train_score,test_score
1,5,16,2,0.460522,0.569242
2,5,32,2,0.460522,0.569242
3,5,4,4,0.460522,0.569242
4,5,16,4,0.460522,0.569242
5,5,32,4,0.460522,0.569242
6,5,4,6,0.460522,0.569242
7,5,16,6,0.460522,0.569242
8,5,32,6,0.460522,0.569242
0,5,4,2,0.460610,0.569198


In [8]:
pd.Panel(predictions).to_pickle("dataset/keras_vae_dense_trained_seperately_nsl_kdd_predictions.pkl")
scores.to_pickle("dataset/keras_vae_dense_trained_seperately_nsl_kdd_scores.pkl")

In [7]:
pd.Panel(predictions)

<class 'pandas.core.panel.Panel'>
Dimensions: 9 (items) x 22544 (major_axis) x 2 (minor_axis)
Items axis: 5_16_2 to 5_4_6
Major_axis axis: 0 to 22543
Minor_axis axis: Attack_prob to Normal_prob